# George D. Crochiere - 0961739
## EE361 - Project 2
## 03/25/2024

In [30]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding
!pip install chainer
import cupy
# Note: if you cannot connect to GPU, then import cupy will not work

# from google.colab import drive
# drive.mount('/content/drive')
# Allow connection to Google Drive to save and access images


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Double Image

In [17]:
import cupy as cp
from PIL import Image

# CUDA kernel code
double_kernel = cp.RawKernel(r'''
extern "C" __global__
void doubleImg(const unsigned char* old_im, unsigned char* new_im, int width, int height)
{
    int tid_x = blockDim.x * blockIdx.x + threadIdx.x;
    int tid_y = blockDim.y * blockIdx.y + threadIdx.y;

    for (int i = 0; i < 3; i++) {
        for (int j = 0; j < 3; j++) {
            int x = tid_x * 3 + i;
            int y = tid_y * 3 + j;

            if (x < width && y < height) {
                int tid = y * width * 3 + x * 3;
                int tid2ds[] = [
                    tid,
                    (y + 1) * width * 3 + (x) * 3,
                    (y) * width * 3 + (x + 1) * 3,
                    (y + 1) * width * 3 + (x + 1) * 3
                ];

                for (int k = 0; k < 3; k++) {
                    unsigned char c = old_im[tid + k];

                    for (int tid2d : tid2ds) {
                        new_im[tid2d + k] = c;
                    }
                }
            }
        }
    }
}

''', 'doubleImg')

def makeDouble(imageFile):
    # Load image using PIL
    oldImage = Image.open(imageFile).convert('RGB')
    width, height = oldImage.size

    # Convert image to cupy array
    oldIm = cp.asarray(oldImage, dtype=cp.ubyte)
    newIm = cp.empty_like(oldIm)

    # Reshape arrays for easier parallelization
    size = width * height
    oldIm = cp.reshape(oldIm, (size, 3),    newIm = cp.reshape(newIm, (size * * 2,))

    # Launch CUDA kernel
    block_dim = (32, 32)
    grid_dim = (int (cp.ceil(width/3/block_dim[0])), int (cp.ceil(height/3/block_dim[1])))

    t3 = time.perf_counter()
    double_kernel((grid_dim), (block_dim), (oldIm, newIm, width, height))
    t4 = time.perf_counter()
    print('time taken to run:',t4-t3)

    # Reshape newIm array back to image dimensions
    newIm = cp.reshape(newIm, (he * 2ight, w * 2idth, 3))

    # Convert cupy array to PIL Image
    newImage = Image.fromarray(cp.asnumpy(newIm))

    # Save the image
    newImage.save('./Dole1D.jpg')

import time
if __name__ == '__main__':
    t1 = time.perf_counter()
    makeDouble('./blue-forest-minimal-4k-kz.jpg')
    t2 = time.perf_counter()
    print('time taken to run:',t2-t1)


SyntaxError: invalid syntax (4086832430.py, line 51)

# Vertical Flip

In [37]:
import cupy
import time
from PIL import Image

# CUDA Kernel - Vertical Flip
verticalFlip_kernel = cp.RawKernel(r'''
extern "C" __global__
void verticalFlip(const unsigned char* old_im, unsigned char* new_im, int width, int height) {
    int tid_x = blockDim.x * blockIdx.x + threadIdx.x;
    int tid_y = blockDim.y * blockIdx.y + threadIdx.y;
    
    for (int i = 0; i < 3; i++) {
        for (int j = 0; j < 3; j++) {
            int x = tid_x * 3 + i;
            int y = tid_y * 3 + j;

            if (x < width && y < height) {
                int tid = y * width * 3 + x * 3;

                unsigned char r = old_im[tid];
                unsigned char g = old_im[tid + 1];
                unsigned char b = old_im[tid + 2];
                
                int tid_new = (y * width * 3) + ((width - x) * 3);
                
                new_im[tid_new] = r;
                new_im[tid_new + 1] = g;
                new_im[tid_new + 2] = b;
            }
        }
    }
}
''', 'verticalFlip')

def makeVerticalFlip(imgFile):
    # Original Image
    oldImage = Image.open(imgFile).convert('RGB')
    width, height = oldImage.size
    size = width * height
    
    # Get image arrays and reshape
    oldImArr = cp.asarray(oldImage, dtype=cp.ubyte)
    newImArr = cp.empty_like(oldImArr)
    oldImArr = cp.reshape(oldImArr, (size, 3))
    newImArr = cp.reshape(newImArr, (size, 3))
    
    # CUDA config
    block_dim = (32,32)
    grid_dim = (int (cp.ceil(width / 3 / block_dim[0])), int(cp.ceil(height / 3 / block_dim[1])))
    
    # Start and run
    t1Vertical = time.perf_counter()
    verticalFlip_kernel((grid_dim), (block_dim), (oldImArr, newImArr, width, height))
    t2Vertical = time.perf_counter()
    print('time taken to run kernel:', t2Vertical - t1Vertical)
    
    # Reshape and convert
    newImArr = cp.reshape(newImArr, (height, width, 3))
    newImage = Image.fromarray(cp.asnumpy(newImArr))
    newImage.save('./VerticalFlipResult.jpg')
    
# Main Method
if __name__ == '__main__':
    tVerticalStart = time.perf_counter()
    makeVerticalFlip('./landscape-scenery-minimalist-4k-r6-2560x1440.jpg');
    tVerticalStop = time.perf_counter()
    print('time taken to run:', tVerticalStop - tVerticalStart)
    

time taken to run kernel: 3.990000004705507e-05
time taken to run: 0.10607509999954345


# Edge Detection

In [ ]:
import cupy as cp
from PIL import Image

# CUDA kernel code
edge_kernel = cp.RawKernel(r'''
extern "C" __global__
void edgeDetection(const unsigned char* old_im, unsigned char* new_im, int size)
{
    
}

''', 'edgeDetection')
